In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from openai import AzureOpenAI 
from dotenv import load_dotenv
import os


In [2]:

load_dotenv()

# ✅ Step 3: Fetch environment variables from the system
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    api_version=azure_api_version
)


In [3]:

class ChatbotState(TypedDict):
    question:str
    answer:str

In [4]:
def Chat_resp(state: ChatbotState) -> ChatbotState:
   
    question = state['question']
    prompt = f"Please provide a short and clear answer related to: {question}"

    try:
        # Azure OpenAI chat completion request
        response = client.chat.completions.create(
            model=azure_deployment,  # Deployment name from Azure
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=150
        )

        # Response text extract kar rahe hain
        answer = response.choices[0].message.content
        state['answer'] = answer

    except Exception as e:
        # Agar koi error aaye toh usse state mein store kar lo
        state['answer'] = f"Error: {str(e)}"

    return state
    

In [9]:
graph=StateGraph(ChatbotState)

graph.add_node("Chat_resp",Chat_resp)

graph.add_edge(START,"Chat_resp")
graph.add_edge("Chat_resp",END)

workflow=graph.compile()

workflow.invoke({"question":"Tell me about Inosuke?"})

{'question': 'Tell me about Inosuke?',
 'answer': 'Inosuke Hashibira is a prominent character from *Demon Slayer: Kimetsu no Yaiba*. He is a brash and aggressive Demon Slayer known for wearing a boar mask and dual-wielding serrated swords. Raised in the wild, he has a feral personality, exceptional physical abilities, and uses the Beast Breathing combat style. Despite his rough demeanor, he shows moments of loyalty and growth throughout the series.'}